# <ins>**Data Wrangling**</ins>

---

In [1]:
# Required Imports
import pandas as pd
import json
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

### **Salary Calculation based on the most relevant job match through vectorization**

In [2]:
df_job_postings = pd.read_csv("../data/transformed/job_postings_cleaned.csv")

df_job_postings["industry"] = df_job_postings["industry"].apply(json.loads)
df_job_postings["speciality"] = df_job_postings["speciality"].apply(json.loads)
df_job_postings["benefits"] = df_job_postings["benefits"].apply(json.loads)

In [3]:
pd.set_option("display.max_columns", None)
df_job_postings.head()

,job_id,job_posting_url,company_id,name,country,state,city,title,description,formatted_experience_level,work_type,remote_allowed,min_salary,max_salary,normalized_salary,currency,pay_period,views,applies,listed_time,original_listed_time,expiry,url,industry,speciality,benefits,employee_count,follower_count,time_recorded
0,921716,https://www.linkedin.com/jobs/view/921716/?trk...,2774458,CORCORAN SAWYER SMITH,US,NJ,JERSEY CITY,MARKETING COORDINATOR,Job descriptionA leading real estate firm in N...,OTHER,FULL_TIME,0,17.0,20.0,38480.0,USD,HOURLY,20,2.0,2024-04-17 23:45:08,2024-04-17 23:45:08,2024-05-17 23:45:08,https://www.linkedin.com/company/corcoran-sawy...,"[""REAL ESTATE""]","[""REAL ESTATE"", ""NEW DEVELOPMENT""]",[],402,2351,1970-01-20 19:56:49.275
1,10998357,https://www.linkedin.com/jobs/view/10998357/?t...,64896719,THE NATIONAL EXEMPLAR,US,OHIO,MARIEMONT,ASSITANT RESTAURANT MANAGER,The National Exemplar is accepting application...,OTHER,FULL_TIME,0,45000.0,65000.0,55000.0,USD,YEARLY,8,0.0,2024-04-16 14:26:54,2024-04-16 14:26:54,2024-05-16 14:26:54,https://www.linkedin.com/company/the-national-...,"[""RESTAURANTS""]","""[OTHER]""",[],15,40,1970-01-20 19:54:39.321
2,23221523,https://www.linkedin.com/jobs/view/23221523/?t...,766262,"ABRAMS FENSTERMAN, LLP",US,NEW YORK,LAKE SUCCESS,SENIOR ELDER LAW / TRUSTS AND ESTATES ASSOCIAT...,Senior Associate Attorney - Elder Law / Trusts...,OTHER,FULL_TIME,0,140000.0,175000.0,157500.0,USD,YEARLY,16,0.0,2024-04-12 04:23:32,2024-04-12 04:23:32,2024-05-12 04:23:32,https://www.linkedin.com/company/abrams-fenste...,"[""LAW PRACTICE""]","[""CIVIL LITIGATION"", ""CORPORATE & SECURITIES L...",[401(K)],222,2427,1970-01-20 19:48:17.299
3,91700727,https://www.linkedin.com/jobs/view/91700727/?t...,1481176,DOWNTOWN RALEIGH ALLIANCE,US,NORTH CAROLINA,RALEIGH,ECONOMIC DEVELOPMENT AND PLANNING INTERN,Job summary:The Economic Development & Plannin...,OTHER,INTERNSHIP,0,14.0,20.0,35360.0,USD,HOURLY,9,4.0,2024-04-18 16:01:39,2024-04-18 16:01:39,2024-05-18 16:01:39,https://www.linkedin.com/company/downtownralei...,"[""NON-PROFIT ORGANIZATIONS""]","[""ECONOMIC DEVELOPMENT"", ""CLEAN & SAFE AMBASSO...",[],22,7825,1970-01-20 19:57:39.851
4,103254301,https://www.linkedin.com/jobs/view/103254301/?...,81942316,RAW CEREAL,US,CA,LOS ANGELES,PRODUCER,Company DescriptionRaw Cereal is a creative de...,OTHER,CONTRACT,1,60000.0,300000.0,180000.0,USD,YEARLY,7,1.0,2024-04-11 18:43:39,2024-04-11 18:43:39,2024-05-11 18:43:39,https://www.linkedin.com/company/raw-cereal,"[""DESIGN SERVICES""]","""[OTHER]""",[],11,447,1970-01-20 19:47:41.202


In [4]:
# Combine relevant features into a single text column
df_job_postings["combined_features"] = df_job_postings.apply(
    lambda x: f"{x['title']} {x['country']} {x['name']} {x['formatted_experience_level']} {x['work_type']}",
    axis=1,
)

In [5]:
# Create TF-IDF matrix from combined features
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_job_postings['combined_features'])

In [6]:
def find_similar_job(index, tfidf_matrix, df):
    cosine_similarities = cosine_similarity(tfidf_matrix[index], tfidf_matrix).flatten()
    similar_indices = cosine_similarities.argsort()[:-50:-1]
    for i in similar_indices:
        if pd.notna(df.iloc[i]['normalized_salary']) and df.iloc[i]['normalized_salary'] > 0:
            return df.iloc[i]['normalized_salary']
    return None

In [7]:
# Fill missing salaries
for index, row in df_job_postings[(df_job_postings['normalized_salary']==0 )|(df_job_postings['normalized_salary'].isnull())].iterrows():
    if pd.isna(row['normalized_salary']) or row['normalized_salary'] == 0:
        similar_salary = find_similar_job(index, tfidf_matrix, df_job_postings)
        df_job_postings.at[index, 'salary_calc'] = similar_salary

In [8]:
df_job_postings['salary_calc'].isna().value_counts()

salary_calc
False    77831
True     43201
Name: count, dtype: int64

In [9]:
df_job_postings['normalized_salary'].isna().value_counts()

normalized_salary
False    121032
Name: count, dtype: int64

In [10]:
df_job_postings['normalized_salary'] = df_job_postings.apply(lambda row: row['salary_calc'] if row['normalized_salary'] == 0 else row['normalized_salary'], axis=1)

### **Text Analysis of Job Description to derive experience and work hours**

In [11]:
def extract_experience(job_description, identifier):
    """
    Extract numerical values for experience or work hours from a job description.

    Parameters:
    - job_description (str): The text of the job description.
    - identifier (str): Specifies what to extract ('experience' or 'workhours').

    Returns:
    - int: The maximum extracted value below a threshold, or 0 if no valid values are found.
    """
    # Define patterns for experience and work hours
    patterns = {
        'experience': [
            r"(?<!Salary: )Experience: .*?(\d+) years?",
            r"(?<!Salary: )(\d+)\+? years? of experience",
            r"(?<!Salary: )(\d+) to (\d+) years of .*? experience"
        ],
        'workhours': [
            r'\b(\d+)\s+hours?\s+per\s+week\b',
            r'\b work .*?(\d+)\s+hours'
        ]
    }
    thresholds = {
        'experience': 30,  # Maximum valid years of experience
        'workhours': 100   # Maximum valid hours per week
    }
    
    # Validate the identifier
    if identifier not in patterns:
        raise ValueError("Invalid identifier. Use 'experience' or 'workhours'.")
    
    # Compile the relevant patterns
    selected_patterns = patterns[identifier]
    threshold = thresholds[identifier]
    
    extracted_values = []
    # Extract matches from the job description
    for pattern in selected_patterns:
        matches = re.findall(pattern, job_description, re.IGNORECASE)
        for match in matches:
            if isinstance(match, tuple):  # Handle ranges like "3 to 5 years"
                extracted_values.extend(int(num) for num in match if num.isdigit())
            else:
                if match.isdigit():
                    extracted_values.append(int(match))

    # Filter and return the maximum valid value below the threshold
    valid_values = [val for val in extracted_values if val < threshold]
    return max(valid_values, default=0)

In [12]:
df_job_postings['experience_derived'] = df_job_postings['description'].astype(str).apply(lambda description: extract_experience(description,'experience'))
df_job_postings['workhours_derived'] = df_job_postings['description'].astype(str).apply(lambda description: extract_experience(description,'workhours'))

### **Currency Conversion - <ins>ARCHIVED</ins>**

Below code has been archived since the column - **normalized_salary** is doing currency conversion already

In [13]:
# from currency_converter import CurrencyConverter

# def salary_calc(period,pay,currency):
#   if currency != 'USD':
#         if currency == 'BBD':
#           pay = pay * 0.50
#         else:
#           c = CurrencyConverter()
#           pay = c.convert(pay, currency, 'USD')
#   if period =='HOURLY':
#     return pay * 8 *5*52
#   elif period == 'WEEKLY':
#     return pay * 52
#   elif period == 'BIWEEKLY':
#     return pay * 26
#   elif period == 'MONTHLY':
#     return pay * 12
#   elif period == 'YEARLY':
#     return pay


# # Fill null currency values based on location
# df_job_postings['currency'] = df_job_postings.groupby('location')['currency'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
# df_job_postings['currency'] = df_job_postings['currency'].fillna('USD')



# # We can try to classify missing values by estimating which pay period is most likely
# def impute_pay_period(row, salary_to_pay_period_map):
#     # Find the closest salary match from known pay_periods
#     min_diff = float('inf')
#     best_match = None
#     for pay_period, salary_range in salary_to_pay_period_map.items():
#         if salary_range[0] <= row['med_salary'] <= salary_range[1]:
#             return pay_period
#     return 'Unknown'  # or a fallback category

# # Example: Creating salary-to-pay_period mapping from known ranges (this will need adjustment)
# salary_to_pay_period_map = {
#     'HOURLY': (0, 500),  # Example range for hourly pay
#     'WEEKLY': (500, 3000),  # Example range for weekly pay
#     'MONTHLY': (3000, 10000),  # Example range for monthly pay
#     'YEARLY': (10000, 12000000)  # Example range for yearly pay
# }


# # Impute missing 'pay_period'
# df_job_postings['pay_period'] = df_job_postings.apply(lambda row: impute_pay_period(row, salary_to_pay_period_map) if pd.isna(row['pay_period']) else row['pay_period'], axis=1)


# df_job_postings['salary_calc'] = df_job_postings.apply(lambda row: salary_calc(row['pay_period'],row['med_salary'],row['currency']),axis = 1)
# df_job_postings.head(1)

### **Feature Encoding**

In [14]:
mlb = LabelEncoder()

df_job_postings['work_type_enc'] = mlb.fit_transform(df_job_postings['work_type'])
df_job_postings['formatted_experience_level_enc'] = mlb.fit_transform(df_job_postings['formatted_experience_level'])
df_job_postings['title_enc'] = mlb.fit_transform(df_job_postings['title'])
df_job_postings['country_enc'] = mlb.fit_transform(df_job_postings['country'])

In [15]:
pd.set_option("display.max_columns", None)
df_job_postings.head()

,job_id,job_posting_url,company_id,name,country,state,city,title,description,formatted_experience_level,work_type,remote_allowed,min_salary,max_salary,normalized_salary,currency,pay_period,views,applies,listed_time,original_listed_time,expiry,url,industry,speciality,benefits,employee_count,follower_count,time_recorded,combined_features,salary_calc,experience_derived,workhours_derived,work_type_enc,formatted_experience_level_enc,title_enc,country_enc
0,921716,https://www.linkedin.com/jobs/view/921716/?trk...,2774458,CORCORAN SAWYER SMITH,US,NJ,JERSEY CITY,MARKETING COORDINATOR,Job descriptionA leading real estate firm in N...,OTHER,FULL_TIME,0,17.0,20.0,38480.0,USD,HOURLY,20,2.0,2024-04-17 23:45:08,2024-04-17 23:45:08,2024-05-17 23:45:08,https://www.linkedin.com/company/corcoran-sawy...,"[""REAL ESTATE""]","[""REAL ESTATE"", ""NEW DEVELOPMENT""]",[],402,2351,1970-01-20 19:56:49.275,MARKETING COORDINATOR US CORCORAN SAWYER SMITH...,NaN,0,0,1,6,34647,76
1,10998357,https://www.linkedin.com/jobs/view/10998357/?t...,64896719,THE NATIONAL EXEMPLAR,US,OHIO,MARIEMONT,ASSITANT RESTAURANT MANAGER,The National Exemplar is accepting application...,OTHER,FULL_TIME,0,45000.0,65000.0,55000.0,USD,YEARLY,8,0.0,2024-04-16 14:26:54,2024-04-16 14:26:54,2024-05-16 14:26:54,https://www.linkedin.com/company/the-national-...,"[""RESTAURANTS""]","""[OTHER]""",[],15,40,1970-01-20 19:54:39.321,ASSITANT RESTAURANT MANAGER US THE NATIONAL EX...,NaN,0,0,1,6,4577,76
2,23221523,https://www.linkedin.com/jobs/view/23221523/?t...,766262,"ABRAMS FENSTERMAN, LLP",US,NEW YORK,LAKE SUCCESS,SENIOR ELDER LAW / TRUSTS AND ESTATES ASSOCIAT...,Senior Associate Attorney - Elder Law / Trusts...,OTHER,FULL_TIME,0,140000.0,175000.0,157500.0,USD,YEARLY,16,0.0,2024-04-12 04:23:32,2024-04-12 04:23:32,2024-05-12 04:23:32,https://www.linkedin.com/company/abrams-fenste...,"[""LAW PRACTICE""]","[""CIVIL LITIGATION"", ""CORPORATE & SECURITIES L...",[401(K)],222,2427,1970-01-20 19:48:17.299,SENIOR ELDER LAW / TRUSTS AND ESTATES ASSOCIAT...,NaN,15,0,1,6,55839,76
3,91700727,https://www.linkedin.com/jobs/view/91700727/?t...,1481176,DOWNTOWN RALEIGH ALLIANCE,US,NORTH CAROLINA,RALEIGH,ECONOMIC DEVELOPMENT AND PLANNING INTERN,Job summary:The Economic Development & Plannin...,OTHER,INTERNSHIP,0,14.0,20.0,35360.0,USD,HOURLY,9,4.0,2024-04-18 16:01:39,2024-04-18 16:01:39,2024-05-18 16:01:39,https://www.linkedin.com/company/downtownralei...,"[""NON-PROFIT ORGANIZATIONS""]","[""ECONOMIC DEVELOPMENT"", ""CLEAN & SAFE AMBASSO...",[],22,7825,1970-01-20 19:57:39.851,ECONOMIC DEVELOPMENT AND PLANNING INTERN US DO...,NaN,0,0,2,6,18835,76
4,103254301,https://www.linkedin.com/jobs/view/103254301/?...,81942316,RAW CEREAL,US,CA,LOS ANGELES,PRODUCER,Company DescriptionRaw Cereal is a creative de...,OTHER,CONTRACT,1,60000.0,300000.0,180000.0,USD,YEARLY,7,1.0,2024-04-11 18:43:39,2024-04-11 18:43:39,2024-05-11 18:43:39,https://www.linkedin.com/company/raw-cereal,"[""DESIGN SERVICES""]","""[OTHER]""",[],11,447,1970-01-20 19:47:41.202,PRODUCER US RAW CEREAL OTHER CONTRACT,NaN,0,0,0,6,44548,76


In [16]:
df_job_postings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121032 entries, 0 to 121031
Data columns (total 37 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   job_id                          121032 non-null  int64  
 1   job_posting_url                 121032 non-null  object 
 2   company_id                      121032 non-null  int64  
 3   name                            121032 non-null  object 
 4   country                         121032 non-null  object 
 5   state                           121032 non-null  object 
 6   city                            121032 non-null  object 
 7   title                           121032 non-null  object 
 8   description                     121032 non-null  object 
 9   formatted_experience_level      121032 non-null  object 
 10  work_type                       121032 non-null  object 
 11  remote_allowed                  121032 non-null  int64  
 12  min_salary      

In [17]:
df_job_postings.columns.to_list()

['job_id',
 'job_posting_url',
 'company_id',
 'name',
 'country',
 'state',
 'city',
 'title',
 'description',
 'formatted_experience_level',
 'work_type',
 'remote_allowed',
 'min_salary',
 'max_salary',
 'normalized_salary',
 'currency',
 'pay_period',
 'views',
 'applies',
 'listed_time',
 'original_listed_time',
 'expiry',
 'url',
 'industry',
 'speciality',
 'benefits',
 'employee_count',
 'follower_count',
 'time_recorded',
 'combined_features',
 'salary_calc',
 'experience_derived',
 'workhours_derived',
 'work_type_enc',
 'formatted_experience_level_enc',
 'title_enc',
 'country_enc']

In [18]:
# final set of columns going forward:
df_job_postings = df_job_postings[
    [
        "job_id",
        "job_posting_url",
        "company_id",
        "name",
        "country",
        "country_enc",
        "state",
        "city",
        "title",
        "title_enc",
        "description",
        "formatted_experience_level",
        "formatted_experience_level_enc",
        "work_type",
        "work_type_enc",
        "remote_allowed",
        "normalized_salary",
        "experience_derived",
        "workhours_derived",
        "currency",
        "views",
        "applies",
        "listed_time",
        "original_listed_time",
        "expiry",
        "url",
        "industry",
        "speciality",
        "benefits",
        "employee_count",
        "follower_count",
        "time_recorded",
        "combined_features",
    ]
]

In [19]:
# Convert lists to JSON strings and save to CSV
df_job_postings["industry"] = df_job_postings["industry"].apply(json.dumps)
df_job_postings["speciality"] = df_job_postings["speciality"].apply(json.dumps)
df_job_postings["benefits"] = df_job_postings["benefits"].apply(json.dumps)

df_job_postings.to_csv("../data/transformed/job_postings_prepared.csv", encoding="utf-8", index=False, header=True)